## Modelo que busca a agenda de setores específicos do MMFDH

In [ ]:
import requests
import csv
import sys
from bs4 import BeautifulSoup as bs
from requests import get
import pandas as pd
import numpy as np
import time
from lxml import html
from tqdm import tqdm
from tqdm.auto import tqdm
import re

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time 
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from requests.exceptions import Timeout

In [2]:
autoridades = 'https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades'
sopa = requests.get(autoridades)
caldeirao = bs(sopa.text, 'lxml')

lista = []

for i in caldeirao.findAll(id="content"):
    a = i.findAll(class_="internal-link")
    for i in a:
        link = i.get('href')
        nome = i.get_text().replace('\xa0','')
        lista.append([nome, link])


# limpa a lista      
lista[16][0] = 'Eduardo Miranda Freire de Melo, Secretário Adjunto de Proteção Global'
del lista[17]
del lista[17]

lista = pd.DataFrame(lista, columns=['Nome_setor', 'link_agenda'])

lista

,Nome_setor,link_agenda
0,"Damares Alves, Ministra de Estado da Mulher, d...",https://www.gov.br/mdh/pt-br/acesso-a-informac...
1,"Sandro Lucio Dezan, Chefe de Gabinete",https://www.gov.br/mdh/pt-br/acesso-a-informac...
2,"Ellen Schelb, Assessora Especial da Ministra",https://www.gov.br/mdh/pt-br/acesso-a-informac...
3,"Henrique Villa da Costa Ferreira, Assessor Esp...",https://www.gov.br/mdh/pt-br/acesso-a-informac...
4,"Marco Vinicius de Carvalho, Assessor Especial ...",https://www.gov.br/mdh/pt-br/acesso-a-informac...
5,"Teresinha de Almeida Ramos Neves,Assessora Esp...",https://www.gov.br/mdh/pt-br/acesso-a-informac...
6,"Rodrigo Rodrigues, Assessor Especial da Ministra",https://www.gov.br/mdh/pt-br/acesso-a-informac...
7,"Sérgio Seabra, Chefe de Assessoria Especial de...",https://www.gov.br/mdh/pt-br/acesso-a-informac...
8,"Fernando Ferreira, Ouvidor Nacional dos Direit...",https://www.gov.br/mdh/pt-br/acesso-a-informac...
9,"Thiago Gonçalves, Chefe da Consultoria Jurídica",https://www.gov.br/mdh/pt-br/acesso-a-informac...


### Link da agenda da Ministra Damares Alves


In [3]:
lista.link_agenda[1]

'https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/chefe-de-gabinete-da-ministra'

In [4]:
# Gera os links com datas da posse até a atualidade

from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

links = []        
start_date = date(2019, 1, 2)
end_date = date(2021, 10, 7)
for i in daterange(start_date, end_date):
        date = lista.link_agenda[0] + '/' + str(i)
        links.append(date)
len(links)

1009

## Raspa a agenda

In [9]:
# Raspa Agenda

salva = links[0].split('/')[7]

dados = []
for j in tqdm(links):
    #contem = re.search('ministra',j) #busca um nome na lista.
    #if contem:
        #pass
    sopa = requests.get(j,timeout=10)
    caldeirao = bs(sopa.text, 'lxml')
    try:
        cargo = caldeirao.find('h1').get_text().strip().replace('Agenda ','')
        nome = caldeirao.find(id="breadcrumbs-current").get_text().split('para')[0].strip().replace('Agenda de ','')
        data = caldeirao.find(id="breadcrumbs-current").get_text().split('para')[1].strip()
        for i in caldeirao.findAll(class_="item-compromisso-wrapper"):
            titulo = i.find('h4').get_text().strip()
            horario = i.find(class_="compromisso-inicio").get_text().strip().replace('h',':')
            local = i.find(class_="compromisso-local").get_text().strip()
            try:
                detalhes = i.find(class_="compromisso-participantes").get_text().replace('Participante(s)\n','').strip()
                agenda = i.find(class_="compromisso-pauta").get_text().replace('Pauta\n','').strip()
            except:
                detalhes = np.nan
                agenda = np.nan
            dados.append([cargo, nome, titulo, data, horario, local, detalhes, agenda])
            #print([cargo, nome, titulo, data, horario, local, detalhes, agenda])
    except Exception:
        err = j
        print(err)
        pass

pd.DataFrame(dados,columns=['cargo', 'nome', 'titulo', 'data', 'horario', 'local', 'detalhes', 'agenda']).to_excel('Agenda_' + salva + '_.xlsx')


https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-01-05
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-01-06
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-01-12
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-01-13
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-01-19
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-01-20
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-01-26
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-01-27
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-02-02
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-02-03
https://www.gov.br/m

https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-09-21
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-09-22
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-09-28
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-09-29
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-10-05
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-10-06
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-10-12
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-10-13
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-10-19
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2019-10-20
https://www.gov.br/m

https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-04-05
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-04-10
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-04-11
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-04-12
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-04-14
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-04-19
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-04-21
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-04-24
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-04-25
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-04-26
https://www.gov.br/m

https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-11-28
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-11-29
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-12-05
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-12-06
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-12-13
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-12-19
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-12-20
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-12-21
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-12-22
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2020-12-23
https://www.gov.br/m

https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2021-07-11
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2021-07-17
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2021-07-18
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2021-07-24
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2021-07-25
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2021-07-31
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2021-08-01
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2021-08-07
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2021-08-08
https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades/agenda-ministra/2021-08-15
https://www.gov.br/m

## Modelo Completo - agenda de todos os setores do ministério

In [ ]:
# pega os links das agendas por integrante

autoridades = 'https://www.gov.br/mdh/pt-br/acesso-a-informacao/agenda-de-autoridades'
sopa = requests.get(autoridades)
caldeirao = bs(sopa.text, 'lxml')

lista = []

for i in caldeirao.findAll(id="content"):
    a = i.findAll(class_="internal-link")
    for i in a:
        link = i.get('href')
        nome = i.get_text().replace('\xa0','')
        lista.append([nome, link])


In [ ]:
# limpa a lista      
lista[16][0] = 'Eduardo Miranda Freire de Melo, Secretário Adjunto de Proteção Global'
del lista[17]
del lista[17]

In [ ]:
# Gera os links com datas até a atualidade

from datetime import timedelta, date

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

links = []        
start_date = date(2019, 1, 2)
end_date = date(2021, 10, 7)
for i in daterange(start_date, end_date):
    for k in lista:
        date = k[1] + '/' + str(i)
        links.append(date)
len(links)

In [ ]:
# Raspa Agenda
dados = []
for j in tqdm(links):
    #contem = re.search('ministra',j) #busca um nome na lista.
    #if contem:
        #pass
    sopa = requests.get(j,timeout=5)
    caldeirao = bs(sopa.text, 'lxml')
    try:
        cargo = caldeirao.find('h1').get_text().strip().replace('Agenda ','')
        nome = caldeirao.find(id="breadcrumbs-current").get_text().split('para')[0].strip().replace('Agenda de ','')
        data = caldeirao.find(id="breadcrumbs-current").get_text().split('para')[1].strip()
        for i in caldeirao.findAll(class_="item-compromisso-wrapper"):
            titulo = i.find('h4').get_text().strip()
            horario = i.find(class_="compromisso-inicio").get_text().strip().replace('h',':')
            local = i.find(class_="compromisso-local").get_text().strip()
            try:
                detalhes = i.find(class_="compromisso-participantes").get_text().replace('Participante(s)\n','').strip()
                agenda = i.find(class_="compromisso-pauta").get_text().replace('Pauta\n','').strip()
            except:
                detalhes = np.nan
                agenda = np.nan
            dados.append([cargo, nome, titulo, data, horario, local, detalhes, agenda])
            #print([cargo, nome, titulo, data, horario, local, detalhes, agenda])
    except Exception:
        pass

pd.DataFrame(dados,columns=['cargo', 'nome', 'titulo', 'data', 'horario', 'local', 'detalhes', 'agenda']).to_excel('Agenda_Gabinete_Damares.xlsx')


In [ ]:
pd.DataFrame(dados,columns=['cargo', 'nome', 'titulo', 'data', 'horario', 'local', 'detalhes', 'agenda']).to_excel('Agenda_Gabinete_Damares.xlsx')
